In [20]:
from hyperopt import hp
from hyperopt.pyll.stochastic import sample
from math import log

In [43]:
# Old-net configuration
min_layers = 1
max_layers = 3

# sets up the neural network
nnets = [None] * (max_layers - min_layers + 1)

for i, num_layers in enumerate(range(min_layers, max_layers + 1)):
    num_hids = [None] * num_layers
    for j in range(num_layers):
        num_hids[j] = hp.qloguniform(
            'num_hid_%i%i' % (i, j), log(10), log(100), 1)

    nnets[i] = num_hids

# define the hyperparamater space to search
hyperspace = {'mln_params': [
    {'arch': hp.choice('arch', nnets)},           
    {'l1_reg': hp.choice(
        'l1_lambda', [None, hp.loguniform('l1_decay', log(1e-5), log(10))])},
    {'l2_reg': hp.choice(
        'l2_lambda', [None, hp.loguniform('l2_decay', log(1e-5), log(10))])},
],
    'optim_params': [
    {'learn_rate': hp.uniform('learn_rate', 0, 1)},
    {'rho': hp.uniform('rho', 0, 1)},
    {'num_epochs': hp.qloguniform(
        'num_epochs', log(10), log(1e4), 1)},
    {'batch_size': hp.quniform('batch_size', 128, 1024, 1)},
    {'init_method': hp.choice(
        'init_method', ['gauss', 'fan-io'])},
    {'scale_factor': hp.uniform(
        'scale_factor', 0, 1)}
]
}
sample(hyperspace)

{'mln_params': ({'arch': (19.0,)},
  {'l1_reg': 0.0022512814271359012},
  {'l2_reg': 1.0362276138845081}),
 'optim_params': ({'learn_rate': 0.8673858561486274},
  {'rho': 0.6542634453039271},
  {'num_epochs': 22.0},
  {'batch_size': 442.0},
  {'init_method': 'fan-io'},
  {'scale_factor': 0.8317983892400194})}

In [58]:
# Force at least 2 layers, cuz we're modern
min_layers = 2
max_layers = 3

# sets up the neural network
nnets = [None] * (max_layers - min_layers + 1)

for i, num_layers in enumerate(range(min_layers, max_layers + 1)):
    num_hids = [None] * num_layers
    for j in range(num_layers):
        num_hids[j] = hp.qloguniform(
            'num_hid_%i%i' % (i, j), log(100), log(1000), 1)

    nnets[i] = num_hids

# define the hyperparamater space to search
hyperspace = ({'mln_params': [
    {'arch': hp.choice('arch', nnets)},
    {'input_p': hp.uniform('ip', 0, 1)},
    {'hidden_p': hp.uniform('hp', 0, 1)},
    {'l1_reg': hp.choice(
        'l1_lambda', [None, hp.loguniform('l1_decay', log(1e-5), log(10))])},
    {'l2_reg': hp.choice(
        'l2_lambda', [None, hp.loguniform('l2_decay', log(1e-5), log(10))])},
],
    'optim_params': [
    {'learn_rate': hp.uniform('learn_rate', 0, 1)},
    {'rho': hp.uniform('rho', 0, 1)},
    {'num_epochs': hp.qloguniform(
        'num_epochs', log(1e2), log(2000), 1)},
    {'batch_size': hp.quniform('batch_size', 128, 1024, 1)},
    {'scale_factor': hp.uniform(
        'scale_factor', 0, 1)}
]
})
sample(hyperspace)

{'mln_params': ({'arch': (911.0, 197.0)},
  {'input_p': 0.12245422200965772},
  {'hidden_p': 0.5798463881590056},
  {'l1_reg': None},
  {'l2_reg': 0.0001110013128202787}),
 'optim_params': ({'learn_rate': 0.6771010571012288},
  {'rho': 0.05746754759777928},
  {'num_epochs': 913.0},
  {'batch_size': 740.0},
  {'scale_factor': 0.2111965350519508})}